In [ ]:
import pandas as pd
import numpy as np
import solver 
from scipy.special import erf, erfinv
import matplotlib.pyplot as plt
import dask
from dask import delayed

def ensemble(planet, fiducial_impact, variables, radians=False):
    samples = 10
    rmin, rmax = 8, 12 
    Y_min, Y_max = 1e3, 10e6
    p_m, stress_p = 3000, 1000
    probabilities = np.linspace(0,1,samples)
    P = np.linspace(0.00001, 0.99999, samples)
    
    radius = np.full(samples,fiducial_impact['radius'])
    angle = np.full(samples,fiducial_impact['angle'])
    strength = np.full(samples,fiducial_impact['strength'])
    velocity = np.full(samples,fiducial_impact['velocity'])
    density = np.full(samples,fiducial_impact['density'])

    var_rad = probabilities * (rmax - rmin) + rmin
    var_ang = np.arccos(np.sqrt(1-probabilities))
    var_ang[var_ang<1] = 1 #not allowing 0 deg 
    var_str = 10** (P * (np.log10(Y_max/Y_min)) + np.log10(Y_min))
    var_den = erfinv(P*2-1)*stress_p*np.sqrt(2)+p_m
    
    
    columns = []
    for var in variables:
        if var == 'radius' or radius.any() <= 0:
            radius = np.random.choice(var_rad, size=samples)
            columns.append(radius)
        if var == 'angle':
            angle = np.random.choice(var_ang, size=samples)
            columns.append(angle)
        if var == 'strength':
            strength = np.random.choice(var_str, size=samples)
            columns.append(strength)
        if var == 'velocity':
            #velocity = np.random.choice(var_vlc, size=samples)
            inf_velocity = np.array([inverse_F(u,11) for u in probabilities])
            v_escape = 11.2
            velocity = np.sqrt(v_escape**2 +inf_velocity**2)*1e3
            columns.append(velocity)
        if var == 'density':
            #density = np.random.choice(var_den, size=samples)
            density = np.random.choice(3000, 10000, samples)
            density[density<1] = 1
            columns.append(density)


    # Ensemble function
    outcome = []
    for i in range(samples):
        #print(angle[i])
        output = delayed(planet.impact)(radius=radius[i], angle=angle[i], strength=strength[i], velocity=velocity[i], density=density[i], init_altitude= 100000, dt = 0.05)
        outcome.append(output[1])
        #print(done)

    outputs = dask.compute(*outcome)
    print(outputs)

    results = []

    for i in range(samples):
        try:
            results.append(outputs[i]['brust_altitude'])
        except KeyError:
            results.append(0)
    
    distribution = pd.DataFrame()
    for i in range(len(variables)):
        distribution[variables[i]] = columns[i]
    
    distribution['Burst Altitude'] = results

    return distribution

def F(x, a):
    return  erf(x/(a*np.sqrt(2)))-(x/a)*np.exp(-x**2/(2*a**2))*np.sqrt(2/np.pi)
def inverse_F(p, a):
    candidates = np.linspace(0, 500, 10000)
    for x in candidates:
        if F(x, a) >= p:
            return x 
    return 500

#testing
if __name__ == '__main__':
    earth = solver.Planet()
    result = ensemble(earth, {'radius': 10, 'angle': 45, 'strength': 1e5, 'velocity': 20e3, 'density': 3000}, variables=['radius','angle','density','velocity','strength'])
    altitudes = result['Brust Altitude']
    count, bins, ignored = plt.hist(altitudes, 40, facecolor ='green')
    print(result)

In [ ]:
#Radius
plt.hist(var_rad,20, facecolor='yellow')
plt.ylabel('b')
plt.xlabel('a')

plt.axis([0, 20, 0, 100])
plt.grid(True)
plt.show(block = False)

#Angle
plt.hist(var_ang, 20, facecolor='yellow')
plt.ylabel('b')
plt.xlabel('a')

plt.axis([0, 2, 0, 100])
plt.grid(True)
plt.show(block = False)

#Strength
plt.hist(np.log10(var_str), 20, facecolor='black')
plt.ylabel('b')
plt.xlabel('a')

plt.axis([0, 8, 0, 100])
plt.grid(True)
plt.show(block = False)

#Denisity
plt.hist(var_den, 40, facecolor='black')
plt.ylabel('b')
plt.xlabel('a')

plt.axis([0, 6000, 0, 100])
plt.grid(True)
plt.show(block = False)

#Velocity
plt.hist(np.log10(var_vlc), 20, facecolor='black')
plt.ylabel('b')
plt.xlabel('a')

plt.axis([0, 8, 0, 100])
plt.grid(True)
plt.show(block = False)